In [98]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.options.display.max_rows = 100

## Feature engineering

In [99]:
# Creating the datasets
house_data_train = pd.read_csv("train.csv")
house_data_test = pd.read_csv("test.csv")
# Target prices:
y = house_data_train['SalePrice']
house_data_train = house_data_train.drop(columns=['SalePrice'])
house_data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [100]:
# Values and meanings from the first column, MSSubClass
MSSubClass_dict = { 
        20:	'1-STORY 1946 & NEWER ALL STYLES', 
        30:	'1-STORY 1945 & OLDER', 
        40:	'1-STORY W/FINISHED ATTIC ALL AGES',
        45:	'1-1/2 STORY - UNFINISHED ALL AGES', 
        50:	'1-1/2 STORY FINISHED ALL AGES', 
        60:	'2-STORY 1946 & NEWER', 
        70:	'2-STORY 1945 & OLDER', 
        75:	'2-1/2 STORY ALL AGES', 
        80:	'SPLIT OR MULTI-LEVEL', 
        85:	'SPLIT FOYER', 
        90:	'DUPLEX - ALL STYLES AND AGES', 
       120:	'1-STORY PUD (Planned Unit Development) - 1946 & NEWER', 
       150:	'1-1/2 STORY PUD - ALL AGES', 
       160:	'2-STORY PUD - 1946 & NEWER', 
       180:	'PUD - MULTILEVEL - INCL SPLIT LEV/FOYER', 
       190:	'2 FAMILY CONVERSION - ALL STYLES AND AGES'}

# Replacing numbers with column names
house_data_train['MSSubClass'].replace(MSSubClass_dict, inplace=True)
house_data_test['MSSubClass'].replace(MSSubClass_dict, inplace=True)

In [101]:
categorical_columns = list(house_data_train.dtypes[house_data_train.dtypes == np.object].index)
numerical_columns = list(house_data_train.dtypes[house_data_train.dtypes != np.object].index)

In [102]:
# Numerical and categorical dataframes
numerical_train = house_data_train[numerical_columns]
numerical_test = house_data_test[numerical_columns]
categorical_train = house_data_train[categorical_columns]
categorical_test = house_data_test[categorical_columns]

In [103]:
# Meanings of the Condition1 and Condition2 column values
cond = {
    'Artery':	'Adjacent to arterial street',
    'Feedr':	'Adjacent to feeder street',	
    'Norm':	'Normal',	
    'RRNn':	'Within 200 of North-South Railroad',
    'RRAn':	'Adjacent to North-South Railroad',
    'PosN':	'Near positive off-site feature--park, greenbelt, etc.',
    'PosA':	'Adjacent to postive off-site feature',
    'RRNe':	'Within 200 of East-West Railroad',
    'RRAe':	'Adjacent to East-West Railroad'
}
# Getting the full name of the conditions
categorical_train['Condition1'].replace(cond, inplace=True)
categorical_train['Condition2'].replace(cond, inplace=True)
categorical_test['Condition1'].replace(cond, inplace=True)
categorical_test['Condition2'].replace(cond, inplace=True)

/usr/local/lib/python3.8/site-packages/pandas/core/generic.py:6610: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [104]:
def sum_1hots(df, column1, column2):
    '''This function is for getting only one hot encoder for repeated 
    conditions in some columns in the dataset '''
    c1 = pd.get_dummies(df[column1])
    c2 = pd.get_dummies(df[column2])
    # Sum both one hot encoders. Exploring this data, I found some 2's, that makes 
    # no sense, so, I replaced them with 1's
    c1c2 = c1.add(c2, fill_value=0).replace(2,1)
    df = df.drop(columns=[column1, column2])
    return df, c1c2

In [105]:
categorical_train, cond_col_train = sum_1hots(categorical_train, 'Condition1', 'Condition2')
categorical_train, ext_col_train = sum_1hots(categorical_train, 'Exterior1st', 'Exterior2nd')
categorical_test, cond_col_test = sum_1hots(categorical_test, 'Condition1', 'Condition2')
categorical_test, ext_col_test = sum_1hots(categorical_test, 'Exterior1st', 'Exterior2nd')

### Handling zero values

In [106]:
categorical_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 40 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   MSSubClass     1460 non-null   object
 1   MSZoning       1460 non-null   object
 2   Street         1460 non-null   object
 3   Alley          91 non-null     object
 4   LotShape       1460 non-null   object
 5   LandContour    1460 non-null   object
 6   Utilities      1460 non-null   object
 7   LotConfig      1460 non-null   object
 8   LandSlope      1460 non-null   object
 9   Neighborhood   1460 non-null   object
 10  BldgType       1460 non-null   object
 11  HouseStyle     1460 non-null   object
 12  RoofStyle      1460 non-null   object
 13  RoofMatl       1460 non-null   object
 14  MasVnrType     1452 non-null   object
 15  ExterQual      1460 non-null   object
 16  ExterCond      1460 non-null   object
 17  Foundation     1460 non-null   object
 18  BsmtQual       1423 non-null

In [107]:
# Replacing with a string the nan values. While exploring what kind of nan values we have in the data, 
# is ok to fill with a Not Aplicable condition. (no basement, no pool, etc)
categorical_train.fillna('Not Aplicable', inplace=True)
categorical_test.fillna('Not Aplicable', inplace=True)

In [108]:
# With the numerical columns, we only have one that is special, the 
# year when the garage was built, we fill with the average year
avg_garage_year = numerical_train['GarageYrBlt'].mean()
numerical_train['GarageYrBlt'].fillna(avg_garage_year, inplace=True)
numerical_test['GarageYrBlt'].fillna(avg_garage_year, inplace=True)
# The other values can be 0
numerical_train.fillna(0, inplace=True)
numerical_test.fillna(0, inplace=True)

/usr/local/lib/python3.8/site-packages/pandas/core/generic.py:6383: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
/usr/local/lib/python3.8/site-packages/pandas/core/frame.py:5171: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


### Creating and appending the One Hot Encoders for each categorical column

In [109]:
def dummies(df, column):
    '''This function accepts a dataframe and a column name. It will get the 
    one hot encoder for categorical column, then, drop it from the original
    dataframe and cancatenate the one hot encoder'''
    df1 = pd.get_dummies(df[column])
    df = df.drop(columns=[column])
    df = pd.concat([df, df1], axis=1)
    return df

In [110]:
categorical_train, bsmt_col_train = sum_1hots(categorical_train, 'BsmtFinType1', 'BsmtFinType2')
categorical_test, bsmt_col_test = sum_1hots(categorical_test, 'BsmtFinType1', 'BsmtFinType2')


In [111]:
categorical_test['HouseStyle'].value_counts()

1Story    745
2Story    427
1.5Fin    160
SLvl       63
SFoyer     46
2.5Unf     13
1.5Unf      5
Name: HouseStyle, dtype: int64

In [112]:
categorical_train['HouseStyle'].value_counts()

1Story    726
2Story    445
1.5Fin    154
SLvl       65
SFoyer     37
1.5Unf     14
2.5Unf     11
2.5Fin      8
Name: HouseStyle, dtype: int64

In [113]:
# Using the custom function in the left categorical data columns.
for column in categorical_test.columns:
    categorical_train = dummies(categorical_train, column)
    categorical_test = dummies(categorical_test, column)

In [114]:
# Appending the sum of one hot encoders created previously
for sumhot in [bsmt_col_train, cond_col_train, ext_col_train]:
    categorical_train = pd.concat([categorical_train, sumhot], axis=1)

for sumhot in [bsmt_col_test, cond_col_test, ext_col_test]:
    categorical_test = pd.concat([categorical_test, sumhot], axis=1)

In [115]:
# miss_test = [i for i in categorical_train.columns if i not in categorical_test.columns]
# for m in miss_test:
#     try:
#         categorical_train = categorical_train.drop(columns=[m])
#     except:
#         print('in exp')

In [116]:
# miss_test = [i for i in categorical_train.columns if i not in categorical_test.columns]
# miss_test

In [117]:
# miss_train = [i for i in categorical_test.columns if i not in categorical_train.columns]
# for m in miss_train:
#     try:
#         categorical_test = categorical_test.drop(columns=[m])
#     except:
#         print('in exp')

In [118]:
common1 = categorical_train.columns.difference(categorical_test.columns)
common2 = categorical_test.columns.difference(categorical_train.columns)
categorical_train = categorical_train.drop(columns=common1) 
categorical_test = categorical_test.drop(columns=common2)

In [119]:
print(common2)
print(common1)

Index(['1-1/2 STORY PUD - ALL AGES'], dtype='object')
Index(['2.5Fin', 'ClyTile', 'Floor', 'Membran', 'Metal', 'Mix', 'NoSeWa',
       'OthW', 'Other', 'Roll', 'TenC'],
      dtype='object')


In [120]:
d1 = categorical_train.columns.difference(categorical_test.columns)
d2 = categorical_test.columns.difference(categorical_train.columns)
print(d1,d2)

Index([], dtype='object') Index([], dtype='object')


In [121]:
len(categorical_train.columns)

245

In [122]:
len(categorical_test.columns)

247

In [123]:
miss_train = [i for i in categorical_train.columns if i not in categorical_test.columns]
miss_train

[]

In [124]:
dup_test = categorical_test.loc[:,categorical_test.columns.duplicated()].columns
dup_train = categorical_train.loc[:,categorical_train.columns.duplicated()].columns
dup_test

Index(['Grvl', 'Not Aplicable', 'Pave', 'Not Aplicable', 'Not Aplicable', 'Ex',
       'Fa', 'Gd', 'TA', 'Stone', 'Ex', 'Fa', 'Gd', 'Not Aplicable', 'TA',
       'Fa', 'Gd', 'Not Aplicable', 'Po', 'TA', 'Gd', 'Not Aplicable', 'Ex',
       'Fa', 'Gd', 'Po', 'TA', 'Ex', 'Fa', 'Gd', 'Not Aplicable', 'TA', 'Mod',
       'Not Aplicable', 'Sev', 'Ex', 'Fa', 'Gd', 'Not Aplicable', 'Po', 'TA',
       'Not Aplicable', 'Not Aplicable', 'Fa', 'Gd', 'Not Aplicable', 'Po',
       'TA', 'Ex', 'Fa', 'Gd', 'Not Aplicable', 'Po', 'TA', 'N', 'Y', 'Ex',
       'Gd', 'Not Aplicable', 'Not Aplicable', 'Not Aplicable', 'Shed',
       'Not Aplicable', 'Not Aplicable', 'Unf', 'Normal', 'BrkFace', 'CBlock',
       'Stone'],
      dtype='object')

In [126]:
from collections import defaultdict
e = defaultdict(int)
f = defaultdict(int)
for x in dup_train:
    e[x] += 1
for x in dup_test:
    f[x] += 1


In [127]:
e

defaultdict(int,
            {'Grvl': 1,
             'Pave': 1,
             'Not Aplicable': 14,
             'Ex': 8,
             'Fa': 9,
             'Gd': 10,
             'TA': 8,
             'Stone': 2,
             'Po': 5,
             'Mod': 1,
             'Sev': 1,
             'N': 1,
             'Y': 1,
             'Shed': 1,
             'Unf': 1,
             'Normal': 1,
             'BrkFace': 1,
             'CBlock': 1})

In [128]:
f

defaultdict(int,
            {'Grvl': 1,
             'Not Aplicable': 18,
             'Pave': 1,
             'Ex': 7,
             'Fa': 8,
             'Gd': 10,
             'TA': 8,
             'Stone': 2,
             'Po': 5,
             'Mod': 1,
             'Sev': 1,
             'N': 1,
             'Y': 1,
             'Shed': 1,
             'Unf': 1,
             'Normal': 1,
             'BrkFace': 1,
             'CBlock': 1})

### Normalizing the numerical data

In [ ]:
from sklearn import preprocessing
#Train set
xnumerical_train = numerical_train.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
xnumerical_train_scaled = min_max_scaler.fit_transform(xnumerical_train)

# Test set
xnumerical_test = numerical_test.values #returns a numpy array
xnumerical_test_scaled = min_max_scaler.fit_transform(xnumerical_test)
# Filling NaN values with 0
xnumerical_test_scaled[np.isnan(xnumerical_test_scaled)] = 0

### Getting the whole dataset

In [ ]:
processed_train = np.append(xnumerical_train_scaled, categorical_train.values, axis=1)
processed_test = np.append(xnumerical_test_scaled, categorical_test.values, axis=1)

### Correlations 

In [ ]:
# norm_train = pd.DataFrame(data = processed_train, columns=categorical_train.columns.append(numerical_train.columns))
# norm_train = pd.concat([norm_train, y], axis=1)
# norm_train

In [ ]:
# corr_matrix = norm_train.corr()
# (corr_matrix['SalePrice']
# .sort_values(ascending=False)
# .where(((corr_matrix['SalePrice'] < -0.05) | (corr_matrix['SalePrice'] > 0.05)), other=np.nan)
# .dropna()
# )

## Machine Learning Models

In [ ]:
from sklearn import svm

regr = svm.SVR()
regr.fit(processed_train, y.values)
predictions_train = regr.predict(processed_train)
predictions_svm = regr.predict(processed_test)
predictions_svm

In [ ]:
np.mean(predictions_train / y.values)

In [ ]:
import csv
j = 1461
with open('predictions.csv', 'w') as p:
    writer = csv.writer(p)
    writer.writerow(['Id','SalePrice'])
    for pred in predictions:
        writer.writerow([j,pred])
        j += 1



In [ ]:
from sklearn.linear_model import SGDRegressor
sgd = SGDRegressor(max_iter=5000, learning_rate='adaptive')
sgd.fit(processed_train, y.values)
predictions_sgd_train = sgd.predict(processed_train)
# predictions_sgd = sgd.predict(processed_test)
# predictions_sgd

In [ ]:
np.max(predictions_sgd_train / y.values)

In [ ]:
import csv
j = 1461
with open('predictions2.csv', 'w') as p:
    writer = csv.writer(p)
    writer.writerow(['Id','SalePrice'])
    for pred in predictions_sgd:
        writer.writerow([j,pred])
        j += 1

In [ ]:
categorical_columns = list(house_data_train.dtypes[house_data_train.dtypes == np.object])